# Synchrone Verteilung Worker1
Die Konfiguration von den Folgenden Workern ist nahezu identisch. Der einzige Unterschied besteht in der Clusterkonfiguration. Der Vollständigkeit halber wird hier sämtlicher benötigter Code gezeigt, jedoch nicht näher erleutert.

In [ ]:
import os
import json
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.datasets import mnist
import numpy as np

## Clusterkonfiguration
Die Clusterkonfiguration der Worker muss als näcshtes Stattfinden. Dabei werden alle Teilnehmer in die os.environ Umgebungsvariablegeschrieben werden.
Dann können wir die Distribution Strategie bestimmen.

In [ ]:
os.environ['TF_CONFIG'] = json.dumps({
    'cluster': {
        'worker': ['192.168.2.1:12345', '192.168.2.1:12345', '192.168.2.3:12345'] #hier werden ip's und ports der teilnehmenden devices angegeben
    },
    'task': {'type': 'worker', 'index': 0} # hier wird die eigene rolle und index angegeben. Da wir hier den index 0 haben wird dieses device zum chief
})

strategy = tf.distribute.MultiWorkerMirroredStrategy() # MultiWorkerMirroredStrategy zum Synchronen lernen auf 3 devices

## Vorbereitung der Trainingsdaten
In diesem Abschnitt werden die trainings und Testen Daten vorbereitet. In diesem Fall wird beispielhaft der MNist Datensatz heruntergeladen und zu einem tensorflow dataset gemacht.
Zur späteren Verwendung kann man besagten TensorFlow Dataset oder aber auch ein numpy array verwenden.
Die Batchsize ist hierbei pro Worker.

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.shuffle(buffer_size=10000).batch(64)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(64)

## Modelerstellung
Nun kann man sein Modell erstellen. In diesem Beispiel wird ein einfaches NN mit einer hidden Layer erstellt. Zu beachten ist, dass das Model innerhalb von strategy.scope() erstellt und compiled werden musst. 
Zusätzlich müssen die Models auf den verschiedenen Worker natürlich identisch sein um keinen Error zu Produzieren

In [ ]:
with strategy.scope():
    distributed_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

   
    distributed_model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])

## Das Modell trainieren
Zum schluss wird mittels model.fit das erstellte Modell trainiert. Hierbei kann man wie üblich zusätzlich parameter wie z.B. Callbacks oder ein Validation Dataset angeben. 
Nachdem man dies auf allen Workern ausgeführt hat beginnt der Synchrone Trainingsprozess.

In [ ]:
distributed_model.fit(train_dataset, epochs=10, steps_per_epoch=20, validation_data=val_dataset)